In [ ]:
# (c) 2019 The Johns Hopkins University Applied Physics Laboratory LLC (JHU/APL).
# All Rights Reserved. This material may be only be used, modified, or reproduced
# by or for the U.S. Government pursuant to the license rights granted under the
# clauses at DFARS 252.227-7013/7014 or FAR 52.227-14. For any other permission,
# please contact the Office of Technology Transfer at JHU/APL.

# NO WARRANTY, NO LIABILITY. THIS MATERIAL IS PROVIDED “AS IS.” JHU/APL MAKES NO
# REPRESENTATION OR WARRANTY WITH RESPECT TO THE PERFORMANCE OF THE MATERIALS,
# INCLUDING THEIR SAFETY, EFFECTIVENESS, OR COMMERCIAL VIABILITY, AND DISCLAIMS
# ALL WARRANTIES IN THE MATERIAL, WHETHER EXPRESS OR IMPLIED, INCLUDING (BUT NOT
# LIMITED TO) ANY AND ALL IMPLIED WARRANTIES OF PERFORMANCE, MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE, AND NON-INFRINGEMENT OF INTELLECTUAL PROPERTY
# OR OTHER THIRD PARTY RIGHTS. ANY USER OF THE MATERIAL ASSUMES THE ENTIRE RISK
# AND LIABILITY FOR USING THE MATERIAL. IN NO EVENT SHALL JHU/APL BE LIABLE TO ANY
# USER OF THE MATERIAL FOR ANY ACTUAL, INDIRECT, CONSEQUENTIAL, SPECIAL OR OTHER
# DAMAGES ARISING FROM THE USE OF, OR INABILITY TO USE, THE MATERIAL, INCLUDING,
# BUT NOT LIMITED TO, ANY DAMAGES FOR LOST PROFITS.

In [ ]:
# Import necessary modules
import json
from pathlib import Path

import pandas as pd
from l2metrics.report import MetricsReport
from tabulate import tabulate

pd.options.display.float_format = '{:,.2f}'.format
%matplotlib ipympl

In [ ]:
# Configure metrics report
kwargs = {}
kwargs['log_dir'] = 'multi_task'
kwargs['variant_mode'] = 'aware'
kwargs['ste_averaging_method']='metrics'
kwargs['perf_measure'] = 'performance'
kwargs['aggregation_method'] = 'mean'
kwargs['maintenance_method'] = 'both'
kwargs['transfer_method'] = 'both'
kwargs['normalization_method'] = 'task'
kwargs['smoothing_method'] = 'flat'
kwargs['window_length'] = None
kwargs['clamp_outliers'] = True
kwargs['data_range_file'] = None # 'data_range.json'
kwargs['show_eval_lines'] = True
kwargs['do_plot'] = True
kwargs['do_save'] = False
kwargs['do_save_settings'] = False

output_dir = Path('results')
output = 'll_metrics'

# Create output directory if it doesn't exist
if kwargs['do_save'] or kwargs['do_save_settings']:
    output_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# Load data range data for normalization and standardize names to lowercase
if kwargs['data_range_file']:
    with open(kwargs['data_range_file']) as config_json:
        data_range = json.load(config_json)
        data_range = {key.lower(): val for key, val in data_range.items()}
else:
    data_range = None

kwargs['data_range'] = data_range

In [ ]:
# Initialize metrics report
report = MetricsReport(**kwargs)

In [ ]:
# Add noise to log data
# report.add_noise(mean=0.0, std=1.0)

In [ ]:
# Calculate metrics in order of their addition to the metrics list.
report.calculate()

In [ ]:
# Print log summary by LX and EX counts
log_summary_df = report.log_summary()
display(log_summary_df)

print(f'\nTotal number of experiences: {report._log_data.shape[0]}')

In [ ]:
print('Lifetime metrics:')
display(report.lifetime_metrics_df)

print('\nTask metrics:')
display(report.task_metrics_df)

In [ ]:
# Plot performance curves for scenario
if kwargs['do_plot']:
    report.plot(save=kwargs['do_save'], show_eval_lines=kwargs['show_eval_lines'], \
        output_dir=str(output_dir))

In [ ]:
# Plot tasks and their corresponding STE
if kwargs['do_plot']:
    report.plot_ste_data(save=kwargs['do_save'], output_dir=str(output_dir))

In [ ]:
# Save data to feather file
if kwargs['do_save']:
    report.save_data(output_dir=str(output_dir), filename=output)

In [ ]:
# Print block summary
print(tabulate(report.block_info, headers='keys', tablefmt='psql'))

In [ ]:
# Print metrics per regime
print(tabulate(report.regime_metrics_df.fillna('N/A'),
               headers='keys', tablefmt='psql', floatfmt=".2f"))